In [43]:
# This is an exmaple how to use spark sql for computing cosine similarity distance with transpose between two rows
df=spark.read.format("csv").option("header","true").option("inferSchema","true").load("/Users/kaicat/2.csv")
df.show()



+---+---+---+---+---+---+
| id| f1| f2| f3| f4| f5|
+---+---+---+---+---+---+
|  a|  1|  2|  3|  4| -5|
|  b|  1|  2|  3|  4| -5|
+---+---+---+---+---+---+



In [44]:
from pyspark.ml.feature import VectorAssembler

In [45]:
featureCols =list(set(df.columns)-{'id'})

assembler = VectorAssembler().setInputCols(featureCols).setOutputCol("features")
df2 = assembler.transform(df)

In [46]:

df2.show()
df3=df2.select("features")
df3.show()

+---+---+---+---+---+---+--------------------+
| id| f1| f2| f3| f4| f5|            features|
+---+---+---+---+---+---+--------------------+
|  a|  1|  2|  3|  4| -5|[2.0,3.0,1.0,4.0,...|
|  b|  1|  2|  3|  4| -5|[2.0,3.0,1.0,4.0,...|
+---+---+---+---+---+---+--------------------+

+--------------------+
|            features|
+--------------------+
|[2.0,3.0,1.0,4.0,...|
|[2.0,3.0,1.0,4.0,...|
+--------------------+



In [47]:
from pyspark.mllib.util import *
df4=MLUtils.convertVectorColumnsFromML(df3,"features")
df4.show()

+--------------------+
|            features|
+--------------------+
|[2.0,3.0,1.0,4.0,...|
|[2.0,3.0,1.0,4.0,...|
+--------------------+



In [48]:
from pyspark.ml.linalg import *
from pyspark.mllib.linalg.distributed import *
rdd1 =df4.select("features").rdd.map(lambda row: row[0] )
# m=RowMatrix(df3.rdd.map(lambda x: x[0])).toBlockMatrix().transpose().toIndexedRowMatrix()

In [49]:
from pyspark.mllib.linalg.distributed import RowMatrix,CoordinateMatrix,MatrixEntry

mat= RowMatrix(rdd1)
cm=CoordinateMatrix(mat.rows.zipWithIndex().flatMap(lambda x:[MatrixEntry(x[1],j,v) for j,v in enumerate(x[0])]))
mat1=cm.transpose().toRowMatrix()

exact=mat1.columnSimilarities()
exact.entries.collect()

[MatrixEntry(0, 1, 1.0)]